In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
#PIVOT to ask: What am I seeking? What information that I want to know? 

In [3]:
#import pandas testing utility
import pandas.util.testing as tm; tm.N = 3

#Create a unpivoted function
def unpivot(frame):
    N, K = frame.shape
    
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    
    # Return the DataFrame
    return DataFrame(data, columns=['date', 'variable', 'value'])

#Set the DataFrame we'll be using
dframe = unpivot(tm.makeTimeDataFrame())

In [4]:
#kita tidak perlu memikirkan cara membuat DataFrame di atas, karena di luar dari course
#yang perlu kita perhatikan adalah DataFrame di bawah
#di bawah merupakan dframe yang masih dalam bentuk stack
#stack dalam artian masih ada baris yang memiliki value (bukan kolom value) yang sama
#CONTOH: variabel A ada banyak, variabel date 2000-01-04 juga masih banyak

#tabel di  bawah bernama stack data table atau flat data table 
dframe

,date,variable,value
0,2000-01-03,A,2.078215
1,2000-01-04,A,-0.674267
2,2000-01-05,A,1.019447
3,2000-01-03,B,0.983512
4,2000-01-04,B,-0.235657
5,2000-01-05,B,-0.020316
6,2000-01-03,C,0.321611
7,2000-01-04,C,0.696576
8,2000-01-05,C,0.064393
9,2000-01-03,D,-0.227679


In [6]:
#melakukan unstack dari tabel yang diberikan
dframe_piv = dframe.pivot('date','variable','value')

In [7]:
dframe_piv

variable,A,B,C,D
date,,,,
2000-01-03,2.078215,0.983512,0.321611,-0.227679
2000-01-04,-0.674267,-0.235657,0.696576,1.911677
2000-01-05,1.019447,-0.020316,0.064393,-1.145167


## Duplicate DataFrame

In [8]:
dframe2 = DataFrame({'key1': ['A']*2 + ['B']*3, 
                    'key2': [2,2,2,3,3]})
dframe2

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3


In [9]:
#memeriksa apakah setiap baris merupakan duplikasi dari baris sebelumya
#index 1 merupakan True karena baris sebelumnya juga memiliki nilai yang sama 
#index 4 True karena index 3 juga memiliki nilai yang sama 
dframe2.duplicated()

0    False
1     True
2    False
3    False
4     True
dtype: bool

In [10]:
dframe3 = DataFrame({'key1': ['A','B','A'], 
                    'key2': [1,2,1]})
dframe3

,key1,key2
0,A,1
1,B,2
2,A,1


In [12]:
#berlaku juga untuk baris yang terpisah lebih dari satu baris (longkap)
#berarti melihat apakah di baris sebelumnya sudah ada baris tsb
dframe3.duplicated()

0    False
1    False
2     True
dtype: bool

In [13]:
dframe4 = DataFrame({'key1': ['A','B','A','C','A'], 
                    'key2': [1,2,1,3,1]})
dframe4

,key1,key2
0,A,1
1,B,2
2,A,1
3,C,3
4,A,1


In [15]:
#berlaku untuk baris lebih dari satu juga
dframe4.duplicated()

0    False
1    False
2     True
3    False
4     True
dtype: bool

In [16]:
#back to dframe2

In [17]:
dframe2

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3


In [18]:
dframe2.duplicated()

0    False
1     True
2    False
3    False
4     True
dtype: bool

In [20]:
#menghapus nilai yang berduplikat
dframe2.drop_duplicates()

,key1,key2
0,A,2
2,B,2
3,B,3


In [21]:
#melakukan spesifikasi kolom yang mana yang mau didrop

#menghapus duplikat di key1, artinya kalau ada A duaa atau B dua, akan dihapus
#cuma mengambil baris pertama dari para duplikat 
dframe2.drop_duplicates(['key1'])

,key1,key2
0,A,2
2,B,2


In [23]:
#argumen take_last sudah diganti dengan menggunakan argumen keep
#dokumentasi lebih lengkap: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html#pandas.DataFrame.drop_duplicates

#mengambil nilai terakhir dari duplikat yang ada 
#A terakhir dari semua A adalah di index 1
#B terakhir dari semua B adalah di index 4, meskipun nilai key2-nya berbeda
dframe2.drop_duplicates(['key1'], keep='last')

,key1,key2
1,A,2
4,B,3


In [24]:
#keep False tidak menampilkan apa-apa, mengapa? 
dframe2.drop_duplicates(['key1'], keep=False)

,key1,key2


In [27]:
dframe4 = dframe2.copy()
dframe_new = DataFrame({'key1': ['C'], 'key2': [2]})
dframe_new

,key1,key2
0,C,2


In [32]:
dframe4 = pd.concat([dframe4, dframe_new], ignore_index=True)

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3
5,C,2
7,C,2


In [38]:
dframe4

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3
5,C,2
6,C,2
7,C,2


In [39]:
dframe4 = dframe4.drop(6)
dframe4 = dframe4.drop(7)
dframe4

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3
5,C,2


In [40]:
#keep=False melihat baris yang tidak ada duplikasinya
dframe4.drop_duplicates(['key1'], keep=False)

,key1,key2
5,C,2


In [41]:
#bagaimana caranya untuk mengambil index yang lebih spesifik? 
#tidak ada methodnya jadi perlu kita buat sendiri

## Mapping

In [42]:
dframe5 = DataFrame({'city': ['Alma', 'Brian Head', 'Fox Park'], 
                   'altitude': [3158, 3000, 2762]})
dframe5

,city,altitude
0,Alma,3158
1,Brian Head,3000
2,Fox Park,2762


In [50]:
state_map = {'Alma': 'Colorado', 'Fox Park': 'Wyoming', 'Brian Head': 'Utah'}
state_map

{'Alma': 'Colorado', 'Fox Park': 'Wyoming', 'Brian Head': 'Utah'}

In [51]:
#membuat kolom baru dengan nama state dan memasukkan valuenya berupa dictionary bernama state_map dengan menggunakan method map
dframe5['state'] = dframe5['city'].map(state_map)

In [52]:
dframe5

,city,altitude,state
0,Alma,3158,Colorado
1,Brian Head,3000,Utah
2,Fox Park,2762,Wyoming


## Replace

In [53]:
ser1 = Series([1,2,3,4,1,2,3,4])
ser1

0    1
1    2
2    3
3    4
4    1
5    2
6    3
7    4
dtype: int64

In [54]:
#mengganti semua nilai 1 dengan nilai null
ser1.replace(1, np.nan)

0    NaN
1    2.0
2    3.0
3    4.0
4    NaN
5    2.0
6    3.0
7    4.0
dtype: float64

In [55]:
#melakukan multiple replace dengan menggunakan list
ser1.replace([1,4], [100,400])

0    100
1      2
2      3
3    400
4    100
5      2
6      3
7    400
dtype: int64

In [57]:
#bisa menggunakan dictionary juga
#semua nilai 4 diganti dengan nilai null

#parameter inplace bisa
ser1.replace({4:np.nan}, inplace=True)

In [58]:
ser1

0    1.0
1    2.0
2    3.0
3    NaN
4    1.0
5    2.0
6    3.0
7    NaN
dtype: float64